#Загрузка библиотек

In [1]:
pip install tensorflow

In [2]:
!pip install gdown

In [3]:
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd

#Загрузка модели и данных

In [4]:
!git clone https://github.com/TheNastyaSaw/Diplom.git

Cloning into 'Diplom'...
remote: Enumerating objects: 28, done.
remote: Counting objects: 100% (28/28), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 28 (delta 2), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (28/28), 42.37 MiB | 11.78 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [5]:
model2 = load_model('/content/Diplom/Centr.keras')

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


#Тестирование по датасету

In [6]:
test_dir = '/content/Diplom/Здания для проверки с классами'

In [7]:
# Нормализация
test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
# Параметры изображений
img_height, img_width = 512, 384 # Размер изображений
batch_size = 8

In [9]:
# Загрузка тестового набора
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

Found 8 images belonging to 8 classes.


In [10]:
# Оценка модели
loss, accuracy = model2.evaluate(test_generator)
print(f"Точность на малой выборке: {accuracy * 100:.2f}%")

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 1.0000 - loss: 0.0015
Точность на малой выборке: 100.00%


#Тестирование по единичному изображению

In [11]:
# Функция для нормализации под нейросеть
def prepare_image(img_path, target_size=(512, 384)):
    img = image.load_img(img_path, target_size=target_size)  # изменяем размер
    img_array = image.img_to_array(img)  # в массив numpy
    img_array = np.expand_dims(img_array, axis=0)  # добавляем размерность батча

    img_array = img_array / 255.0

    return img_array

In [12]:
img = prepare_image('/content/Diplom/Здания для проверки/1 (0 класс).jpg')  # путь к вашему изображению

In [13]:
predictions = model2.predict(img)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


In [14]:
predicted_class = np.argmax(predictions, axis=1)  # индекс класса с максимальной вероятностью
print("Предсказанный класс:", predicted_class)

Предсказанный класс: [0]


In [15]:
df = pd.read_csv('/content/Diplom/Centr.csv')

In [16]:
df

,nomer,address,coordinates,info,url_maps
0,0,"Советская улица, 3/8, Любим, Ярославская область","58.361711, 40.687412","Два столетия назад, в первой половине девятнад...",https://yandex.ru/maps/20592/lyubim/house/sove...
1,1,"Советская улица, 9/23, Любим, Ярославская область","58.360446, 40.688131","Купеческий род Лобановых, прославившийся успеш...",https://yandex.ru/maps/20592/lyubim/house/sove...
2,2,"Советская улица, 13, Любим, Ярославская область","58.359408, 40.688643","Старое здание, возвышающееся над сонными улочк...",https://yandex.ru/maps/20592/lyubim/house/sove...
3,3,"улица Ленина, 21/4, Любим, Ярославская область","58.360371, 40.687394",На перекрёстке улиц Ленина и Советской стоит з...,https://yandex.ru/maps/20592/lyubim/house/ulit...
4,4,"Красноармейская улица, 1, Любим, Ярославская о...","58.361820, 40.683460","Двухэтажное здание на Красноармейской, с вывес...",https://yandex.ru/maps/20592/lyubim/house/kras...
5,5,"улица Раевского, 4, Любим, Ярославская область","58.362259, 40.689775","В самом сердце Любима, небольшого городка Ярос...",https://yandex.ru/maps/20592/lyubim/house/ulit...
6,6,"улица Раевского, 15, Любим, Ярославская область","58.361811, 40.693682","Поодаль от центра старинного Любима стоит дом,...",https://yandex.ru/maps/20592/lyubim/house/ulit...
7,7,"улица Набережная реки Обноры, 1, Любим, Яросла...","58.364025, 40.697976","На берегу тихой реки, словно старый страж, сто...",https://yandex.ru/maps/20592/lyubim/house/ulit...


In [17]:
df.set_index('nomer', inplace=True)  # делаем 'nomer' индексом

In [18]:
record = df.loc[int(predicted_class)]
dfs = pd.DataFrame(record)

<ipython-input-18-562563110>:1: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  record = df.loc[int(predicted_class)]


In [19]:
dfs

,0
address,"Советская улица, 3/8, Любим, Ярославская область"
coordinates,"58.361711, 40.687412"
info,"Два столетия назад, в первой половине девятнад..."
url_maps,https://yandex.ru/maps/20592/lyubim/house/sove...
